In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

In [10]:

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_url(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '(Sebuah tautan)', text)

def remove_identity(text):
    return re.sub(r'\n(Biro|Bidang|Kabinet|Divisi|Departemen|Himpunan|#|Line|Twitter|Instagram|Facebook|Youtube|Ask fm|Wa|WA|@).*', '\n\n', text)

In [11]:
separator = "<|endoftext|>"

In [12]:
list_caption = []
for file in os.listdir("Bahan Scraping"):
    with open("Bahan Scraping/"+file,  encoding="utf8") as fp:
        soup = BeautifulSoup(fp, 'html.parser')
        list_caption.extend( [i.get('alt') for i in soup.find_all('img') 
                            if re.search(r'\[(.*?)\]', i.get('alt',""))]) 
list_caption = list(dict.fromkeys(list_caption))

In [13]:
df = pd.DataFrame(list_caption, columns=['raw_caption'])


#title is inside bracket
df['title'] = df['raw_caption'].str.extract(r'\[(.*?)\]', expand=False)

#Remove title from caption, remove leading newline, remove emoji
df['content'] = df['raw_caption'].apply(lambda x: deEmojify(x)).apply(lambda x: remove_url(x)).apply(lambda x: remove_identity(x)).str.split(r'\[.*?\]').apply(lambda x: x[-1]).apply(lambda x: x.lstrip()).apply(lambda x : re.sub(r'\n+', '\n',x))

df['processed'] = df.apply(lambda x: x['title']+" - "+x['content'], axis=1)


In [27]:
print(df.sample(1)['processed'].values[-1])

ACTION PLAN PPSMB UGM 2020 - Xayrli kun, Gadjah Mada Muda! (Xayrli kun merupakaan sapaan selamat sore dalam bahasa Uzbek)
Setelah melalui berbagai rangkaian kegiatan pada PPSMB UGM 2020, Gamada akan melaksanakan kegiatan action plan sebagai wadah untuk berkontribusi nyata pada lingkungan sekitar. Pada PPSMB UGM 2020, Action Plan dilakukan oleh masing-masing Gamada pada wilayah tempat tinggal masing-masing.
Selamat kepada Gamada yang telah meraih penghargaaan Action Plan terbaik.
Video:
Juara 1 : Aisya Rahmadani Humaningayu dari Harjono Danoesastro 18
Juara 2 : Maria Ratnasari Anggraini Santoso dari Sardjito 19
Juara 3 : Wahyu Cahyo S. dari Poerbatjaraka 5
Juara Harapan 1 : Hardian Ridho A. dari Notonagoro 3
Juara Harapan 2 : Rajendra Yodha P.S. dari Poerbatjaraka 3
Juara Harapan 3 : Fanesa Dwina dari Kertanegara 6
Esai:
Juara 1 : Ni Made Lalita Gita Gayatri dari Herman Yohanes 17 
Juara 2 : Tjok Istri Sintawati dari Taroeno Djojodiningrat 2
Juara 3 : Ramadi Syarif Hidayatullah dari Wre

In [8]:
df['processed'].to_csv("list_caption.csv", index=False)